In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return [w.lower() for w in words]
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
result = wiki.take(1)[0]

# This chunk of code solves the problem, but it written out in extreme detail

In [ ]:
wiki_keyed = wiki.map(lambda x: (x[0], x[1:]))

In [ ]:
wiki_next = wiki_keyed.map(lambda x: (x[0], zip(x[1], x[1][1:]+[''])))

In [ ]:
wiki_flat = wiki_next.flatMap(lambda x: x[1])

In [ ]:
wiki_targets = wiki_flat.filter(lambda x: x[0] == u'narodnaya')

In [ ]:
res = wiki_targets.map(lambda x: (x[0]+'_'+x[1], ))

In [ ]:
counts = dict(res.countByKey())

In [ ]:
for k in counts.keys():
    print "{}\t {}".format(k, counts[k])

# This chunk of code solves the problem, but in a condensed and cleaned manner

In [2]:
wiki_flat = wiki.flatMap(lambda x: zip(x[1:], x[2:]+['']))

In [3]:
wiki_targets = wiki_flat.filter(lambda x: x[0] == u'narodnaya').map(lambda x: (x[0]+'_'+x[1], ))

In [4]:
counts = dict(wiki_targets.countByKey())
for k in counts.keys():
    print "{}\t {}".format(k, counts[k])

narodnaya_gazeta	 1
narodnaya_volya	 9
